In [1]:
import json
import os
import sys

import numpy as np
import pandas as pd

import sgkit as sg
from sgkit.io.vcf import vcf_to_zarr

import tsinfer
import tskit
import tszip

sys.path.append("../../tsimpute/src/")
import util
import perform_imputation_by_sample_matching as pism


### Download unified genealogies of chr20 q-arm

In [2]:
%%bash
data_dir="../data/unified_genealogies/"
ts_file="hgdp_tgp_sgdp_chr20_p.dated.trees.tsz"

mkdir -p $data_dir
if [ ! -f $data_dir"/"$ts_file ]; then
    wget --quiet -P $data_dir "https://zenodo.org/record/5495535/files/"$ts_file
fi


In [3]:
data_dir="../data/unified_genealogies/"
ts_file="hgdp_tgp_sgdp_chr20_p.dated.trees.tsz"

ts = tszip.decompress(data_dir + ts_file)
ts


### Simplify the trees

In [4]:
# Keep only high-coverage individuals
min_coverage = 30

high_cov_inds = []
for ind in ts.individuals():
    metadata = json.loads(ind.metadata)
    if not "sample" in metadata:
        continue
    is_high_cov = float(metadata["coverage"]) >= min_coverage
    if is_high_cov:
        high_cov_inds.append(ind.id)

print(f"All individuals: {len(ts.individuals())}")
print(f"High-coverage individuals: {len(high_cov_inds)}")


All individuals: 3754
High-coverage individuals: 876


In [5]:
node_ids = np.arange(ts.num_nodes)
keep_nodes = node_ids[np.isin(ts.nodes_individual, high_cov_inds)]
ts_hc = ts.simplify(
    samples=keep_nodes,
    filter_populations=True,
    filter_individuals=True,
    filter_nodes=True,
    filter_sites=True,
)
ts_hc


In [6]:
# Keep only biallelic sites
biallelic_sites = []
for v in ts_hc.variants():
    if len(np.unique(v.genotypes)) == 2:
        biallelic_sites.append(v.site.id)

print(f"All sites: {ts_hc.num_sites}")
print(f"Biallelic sites: {len(biallelic_sites)}")


All sites: 560106
Biallelic sites: 558130


In [7]:
site_ids = np.arange(ts_hc.num_sites)
remove_sites = site_ids[np.isin(site_ids, biallelic_sites, invert=True)]
ts_hc_bi = ts_hc.delete_sites(site_ids=remove_sites)
assert ts_hc_bi.num_sites == len(biallelic_sites)
ts_hc_bi


In [8]:
# Keep only biallelic sites without indels
# Note: No such sites exist in this dataset
indel_sites = []
for site in ts_hc_bi.sites():
    max_allele_len = np.max([len(a) for a in site.alleles])
    if max_allele_len != 1:
        indel_sites.append(site.id)

print(f"All biallelic sites: {ts_hc_bi.num_sites}")
print(f"Biallelic sites with indels: {len(indel_sites)}")


All biallelic sites: 558130
Biallelic sites with indels: 0


In [9]:
out_dir = "../data/trees/"
ts_hc_bi_file = out_dir + "hgdp_tgp_sgdp_chr20_p.dated.hc_bi.trees.tsz"
tszip.compress(ts_hc_bi, ts_hc_bi_file)
